# Example Classification: $G=\mathrm{SU}(7)^a\times\mathrm{SU}(8)^b$

This notebook demonstrates the enumeration of all irreducible cliques with gauge group $\mathrm{SU}(7)^a\times\mathrm{SU}(8)^b$ satisfying the bound $\Delta+28n_-^\mathfrak{g} + T_\text{min} \leq 273$, and with any number of tensors $T$ and any hypermultiplet representations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import colorcet as cc
from tqdm import tqdm

import sys
sys.path.append('./src')

from graph import Graph
from irreducibleClique import unique_cliques

In [ ]:
plt.rcParams['figure.dpi'] = 100
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = '\\usepackage{amssymb} \\usepackage{amsmath}'
colA = '#E06836'
colB = '#4B9423'

## Graph initialization

Start by creating the multigraph with vertices for $A_6\sim\mathrm{SU}(7)$ and $A_7\sim\mathrm{SU}(8)$ out to a maximum value of $\Delta_i\leq 400$.

In [ ]:
Δ_schedule = [
    ['A', [[6, 400], [7, 400]]]
]

graph = Graph(Δ_schedule, progress=True)

Let's visualize the distribution of vertices according to their values of $\Delta_i$ and $b_i\cdot b_i$:

In [ ]:
Δ_bibi_typeA = np.array([[vertex.Δ, vertex.bibi] for vertex in graph.vertices if vertex.type == 'A']).T
Δ_bibi_typeB = np.array([[vertex.Δ, vertex.bibi] for vertex in graph.vertices if vertex.type == 'B']).T

fig, ax = plt.subplots(1, 1, figsize=(8,4))

plt.plot(Δ_bibi_typeA[0], Δ_bibi_typeA[1], '.', c=colA, ms=3, label='type A')
plt.plot(Δ_bibi_typeB[0], Δ_bibi_typeB[1], '.', c=colB, ms=5, label='type B')

plt.axhline(0, lw=0.5, c='k', zorder=0)
plt.axvline(0, lw=0.5, c='k', zorder=0)
plt.axvline(273, ls='--', c='gray', zorder=0)

plt.xlabel('$\Delta_i$')
plt.ylabel('$b_i\cdot b_i$')

plt.legend()

plt.show()

There are exactly eleven type-B vertices $(b_i\cdot b_i\leq 0)$:

In [ ]:
for vertex in graph.vertices:
    if vertex.type == 'B':
        vertex.display()

## Irreducible cliques

Type-A and type-B cliques are build by branching by individual vertices one-by-one. Type-AB cliques are then build by attaching type-B cliques to type-A cliques via only A-B nontrivial edges.

### Type-A irreducible cliques

First construct all type-A irreducible cliques. Even with no pruning related to $\Delta+28n_-^\mathfrak{g}$ the branching quickly peters out at $4$-cliques.

In [ ]:
# type-B cliques organized by k
irreducible_cliques_A = [[], [vertex.get_simple_clique(graph) for vertex in graph.vertices if vertex.type == 'A']]

while len(irreducible_cliques_A[-1]) > 0:
    # Produce (k+1)-cliques from k-cliques
    cliques_branched = []

    for clique in tqdm(irreducible_cliques_A[-1], desc=f'Branching {len(irreducible_cliques_A)-1:2}-cliques'):
        # Augment by one vertex and prune
        cliques_new = [clq for clq in clique.branch_by_vertex() if clq.prune_function(improved=False) <= 300]
        cliques_branched.extend(cliques_new)

    # Remove duplicates and save
    cliques_branched = unique_cliques(cliques_branched)
    irreducible_cliques_A.append(cliques_branched)

# Prepare for adding type-B vertices by changing branching mode
for k_cliques in irreducible_cliques_A:
    for clique in k_cliques:
        clique.branching_mode = 'B'

### Type-B irreducible cliques

Now construct all type-B irreducible cliques, using a fairly crude bound to determine when to prune.<br>
The branch-and-prune process terminates at $(13\sim14)$-cliques.

In [ ]:
# type-B cliques organized by k
irreducible_cliques_B = [[], [vertex.get_simple_clique(graph) for vertex in graph.vertices if vertex.type == 'B']]

while len(irreducible_cliques_B[-1]) > 0:
    # Produce k-cliques from (k-1)-cliques
    cliques_branched = []

    for clique in tqdm(irreducible_cliques_B[-1], desc=f'Branching {len(irreducible_cliques_B)-1:2}-cliques'):
        # Augment by one vertex and prune
        cliques_new = [clq for clq in clique.branch_by_vertex() if clq.prune_function(improved=False) <= 300]
        cliques_branched.extend(cliques_new)

    # Remove duplicates and save
    cliques_branched = unique_cliques(cliques_branched)
    irreducible_cliques_B.append(cliques_branched)

At this point one could record data about type-B cliques which have $f_{1/2} < 0$ to improve the pruning of type-AB cliques below (for these two simple groups there are no cliques with $f_{1/2} < 0$):

In [ ]:
for k_cliques in irreducible_cliques_B:
    for clique in k_cliques:
        graph.update_neg_values(clique)

graph.vertex_neg_values

### Type-AB irreducible cliques

Complete the construction of irreducible cliques by joining type-B cliques to type-A cliques.
First, collect type-B cliques which either have $n_+^\mathfrak{g}=0$ or $n_+^{\overline{\mathfrak{g}}}=0$ ($\overline{\mathfrak{g}}$ denoting $\mathfrak{g}=b_i\cdot b_j$ but with active vertices removed).

In [ ]:
cliques_B_n_pos_0 = []
cliques_B_n_pos_restricted_0 = []

for k_cliques in irreducible_cliques_B:
    for clique in k_cliques:
        if len(clique.active_vertices) == 0:
            continue

        if clique.n_pos_bibj == 0:
            cliques_B_n_pos_0.append(clique)
            continue

        n_pos_restricted = clique.signature_gram_bibj_restricted(clique.active_vertices)[0]
        if n_pos_restricted == 0:
            cliques_B_n_pos_restricted_0.append(clique)

order = np.argsort([clique.f(0.5, False) for clique in cliques_B_n_pos_0])
cliques_B_n_pos_0 = [cliques_B_n_pos_0[ii] for ii in order]

print(len(cliques_B_n_pos_0), len(cliques_B_n_pos_restricted_0))

For each type-A clique, build type-AB cliques separately:

In [ ]:
cliques_AB = []
nA_max = 0
nB_max = len(irreducible_cliques_B)
for k_cliques in irreducible_cliques_A:
    for clique in tqdm(k_cliques):
        cliques_AB.extend(clique.build_AB_cliques(cliques_B_n_pos_0, cliques_B_n_pos_restricted_0))
        nA_max = max(nA_max, clique.num_AB[0])
        nB_max = max(nB_max, clique.num_AB[1])

### Bounded irreducible cliques
Restrict to irreducible cliques satisfying $\Delta+28n_-^\mathfrak{g} + T_\text{min} \leq 273$ and mark all participating vertices as 'used'

In [ ]:
irreducible_cliques_bounded = [[[] for _ in range(nB_max+1)] for _ in range(nA_max+1)]

irreducible_cliques_B_flat = [aa for bb in irreducible_cliques_B for aa in bb]

for clique in [*irreducible_cliques_B_flat, *cliques_AB]:
    nA, nB = clique.num_AB
    if clique.Δ_28n + clique.T_min <= 273:
        irreducible_cliques_bounded[nA][nB].append(clique)

        for vertex in clique.vertices:
            vertex.used = True

maxdigs = int(np.log10(np.max([[len(aa) for aa in row] for row in irreducible_cliques_bounded]))) + 1
print('NA╲NB │  ' + '  '.join([f'{ii:{maxdigs}}' for ii in range(nB_max+1)]))
print(6*'─' + '┼' + ((nB_max+1)*(maxdigs+2))*'─')
for nA, cliques_by_A in enumerate(irreducible_cliques_bounded):
    print(f'{nA:5} │', end='  ')
    for cliques_by_AB in cliques_by_A:
        if len(cliques_by_AB) > 0:
            print(f'{len(cliques_by_AB):{maxdigs}}', end='  ')
        else:
            print(end=(maxdigs+2)*' ')
    print()

Looking at the distribution of 'used' vertices retroactively justifies the choice $\Delta_\text{max}=400$:

In [ ]:
Δ_bibi_used = np.array([[vertex.Δ, vertex.bibi] for vertex in graph.vertices if     vertex.used]).T
Δ_bibi_not  = np.array([[vertex.Δ, vertex.bibi] for vertex in graph.vertices if not vertex.used]).T

fig, ax = plt.subplots(1, 1, figsize=(10,4))

plt.plot(Δ_bibi_used[0], Δ_bibi_used[1], '.', c='g', ms=5,   label='used')
plt.plot(Δ_bibi_not[0],  Δ_bibi_not[1],  '.', c='r', ms=3, label='not')

plt.axvline(273, ls='--', c='gray', zorder=0)

plt.xlabel('$\Delta_i$')
plt.ylabel('$b_i\cdot b_i$')

plt.legend()

plt.show()

### Irreducible clique distributions

In [ ]:
binwidth = 5
data = np.zeros([2, 13, 300//binwidth], dtype='int')
for cliques_by_A in irreducible_cliques_bounded:
    for cliques_by_AB in cliques_by_A:
        for clique in cliques_by_AB:
            data[clique.n_pos_bibj, clique.k, clique.Δ_28n//binwidth] += 1
vmax = np.max(data)

fig, axes = plt.subplots(2, 1, figsize=(10,4), sharex=True)

for data_npos, ax in zip(data, axes):
    im = ax.imshow(data_npos, origin='lower', extent=(0, 300,-0.5,12.5),
                    aspect='auto', cmap='cet_CET_L8', norm=colors.LogNorm(vmax=vmax))
    ax.set_yticks(range(0, 13, 2))
    ax.set_ylabel('$k$')
    ax.set_facecolor('0.9')

axes[0].text(0.03, 0.8, '$n_+^\mathfrak{g} = 0$', transform=axes[0].transAxes, bbox=dict(facecolor='w', alpha=0.9))
axes[1].text(0.03, 0.8, '$n_+^\mathfrak{g} = 1$', transform=axes[1].transAxes, bbox=dict(facecolor='w', alpha=0.9))
plt.xticks([*np.arange(0, 301, 50), 273])
plt.xlabel('$\Delta+28n_-^\mathfrak{g}$')
plt.tight_layout()

plt.colorbar(im, ax=axes.ravel().tolist(), pad=0.02)
plt.show()

From these bounded irreducible cliques one can build anomaly-free theories in a straightforward way using the bound
$$ \sum_\alpha (\Delta+28n_-^\mathfrak{g})(C_\alpha) \leq 273 $$
as a guide.